In [ ]:
# imports
import os
import glob
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# extracting features
def extract_features(signal: np.ndarray) -> dict:
    return {
        "mean": np.mean(signal),
        "std": np.std(signal),
        "var": np.var(signal),
        "rms": np.sqrt(signal),
        "max": np.max(signal),
        "min": np.min(signal),
        "skew": pd.Series(signal).skew(),
        "kurt": pd.Series(signal).kurtosis(),
    }